In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch
import torch.nn as nn
import torch.optim as optim
import os

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18,googlenet,densenet121,mobilenet_v3_small,mobilenet_v3_large
from sklearn.model_selection import train_test_split
from PIL import Image

In [10]:
#does torch use cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
device

device(type='cpu')

In [6]:
def multiple_dir_files_to_df(dirs, label):
    df = pd.DataFrame()
    for idx,dir in enumerate(dirs):
        files = os.listdir(dir)
        files = [dir + '/' + file for file in files]
        df = pd.concat([df,pd.DataFrame({'file': files, 'label': label[idx]})])
    return df

common_path="Image/"

labels=["neutral","happy","sad","fear","angry","surprise","disgust","Happily_Surprised",
"Happily_Disgusted","Sadly_Fearful","Sadly_Angry","Sadly_Surprised","Sadly_Disgusted",
"Fearfully_Angry","Fearfully_Surprised","Fearfully_Disgusted","Angrily_Surprised",
"Angrily_Disgusted","Disgustedly_Surprised","Appalled","Hatred","Awed"]

database=[common_path+label for label in labels]

df = multiple_dir_files_to_df(database,np.linspace(0,len(labels)-1,len(labels)))
X,y=df["file"].tolist(),df["label"].tolist()

In [7]:
# Définir un Dataset personnalisé
class CustomDataset(Dataset):
    def __init__(self, file_names, labels, transform=None):
        self.file_names = file_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = f'{self.file_names[idx]}' 
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label


In [8]:
# Define the image transformations
transform = transforms.Compose([
    transforms.CenterCrop((500, 500)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [9]:
class GoogleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.googlenet = googlenet(pretrained=True)
        self.googlenet.inception4d=nn.Identity()
        self.googlenet.inception4e=nn.Identity()
        self.googlenet.inception5a=nn.Identity()
        self.googlenet.inception5b=nn.Identity()
        self.googlenet.fc=nn.Identity()

    def forward(self, x):
        return self.googlenet(x)

model= GoogleNet()
print(model)

c:\Users\tgeoffroy\Anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tgeoffroy\Anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogleNet(
  (googlenet): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn

In [6]:
# Définir le modèle ResNet
class ResNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(256, 7)
        self.resnet.layer4= nn.Identity()
        self.resnet.layer1[-1].bn2 = nn.Sequential(
            self.resnet.layer1[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer2[-1].bn2 = nn.Sequential(
            self.resnet.layer2[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer3[-1].bn2 = nn.Sequential(
            self.resnet.layer3[-1].bn2,
            nn.Dropout2d(p=0.15)
        )
        self.resnet.fc=nn.Identity()
        
    def forward(self, x):
        return self.resnet(x)

model= ResNetModel()
print(model)

c:\Users\tgeoffroy\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tgeoffroy\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\tgeoffroy/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 101MB/s] 


ResNetModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [7]:
class DenseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.densenet = densenet121(pretrained=True)
        self.densenet.classifier = nn.Identity()

    def forward(self, x):
        return self.densenet(x)
    
print(DenseNet())

c:\Users\tgeoffroy\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tgeoffroy\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\tgeoffroy/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 55.2MB/s]


DenseNet(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0

In [8]:
class MobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = mobilenet_v3_small(weights='IMAGENET1K_V1')
        self.mobilenet.classifier = nn.Identity()

    def forward(self, x):
        return self.mobilenet(x)
        
print(MobileNet())

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\tgeoffroy/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 30.1MB/s]


MobileNet(
  (mobilenet): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
            (activation): ReLU()
            (scale_activation): Hardsigmoid()
          )
          (2): Conv

In [9]:
class MobileNetLarge(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = mobilenet_v3_large(weights='IMAGENET1K_V1')
        self.mobilenet.classifier = nn.Identity()

    def forward(self, x):
        return self.mobilenet(x)
        
print(MobileNetLarge())

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to C:\Users\tgeoffroy/.cache\torch\hub\checkpoints\mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 66.1MB/s]


MobileNetLarge(
  (mobilenet): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          )
        )
      )
      (2): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActi

In [10]:
# Créer les jeux de données et les chargeurs de données
t_dataset = CustomDataset(X, y, transform)
t_loader = DataLoader(t_dataset, batch_size=10, shuffle=False)

In [13]:
#check if gpu is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
torch.cuda.device_count()

0

In [11]:
pretrained_model ="./models/MobileNetLarge_17.pth"
model = MobileNetLarge()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(pretrained_model, map_location=device), strict=False)
model.eval()
df=pd.DataFrame()

for i, (x, y) in enumerate(t_loader):
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        output = model(x)
        df=pd.concat([df,pd.DataFrame(output.cpu().numpy())])
        print(df.shape)

(10, 960)
(20, 960)
(30, 960)
(40, 960)
(50, 960)
(60, 960)
(70, 960)
(80, 960)
(90, 960)
(100, 960)
(110, 960)
(120, 960)
(130, 960)
(140, 960)
(150, 960)
(160, 960)
(170, 960)
(180, 960)
(190, 960)
(200, 960)
(210, 960)
(220, 960)
(230, 960)
(240, 960)
(250, 960)
(260, 960)
(270, 960)
(280, 960)
(290, 960)
(300, 960)
(310, 960)
(320, 960)
(330, 960)
(340, 960)
(350, 960)
(360, 960)
(370, 960)
(380, 960)
(390, 960)
(400, 960)
(410, 960)
(420, 960)
(430, 960)
(440, 960)
(450, 960)
(460, 960)
(470, 960)
(480, 960)
(490, 960)
(500, 960)
(510, 960)
(520, 960)
(530, 960)
(540, 960)
(550, 960)
(560, 960)
(570, 960)
(580, 960)
(590, 960)
(600, 960)
(610, 960)
(620, 960)
(630, 960)
(640, 960)
(650, 960)
(660, 960)
(670, 960)
(680, 960)
(690, 960)
(700, 960)
(710, 960)
(720, 960)
(730, 960)
(740, 960)
(750, 960)
(760, 960)
(770, 960)
(780, 960)
(790, 960)
(800, 960)
(810, 960)
(820, 960)
(830, 960)
(840, 960)
(850, 960)
(860, 960)
(870, 960)
(880, 960)
(890, 960)
(900, 960)
(910, 960)
(920, 96

KeyboardInterrupt: 

In [12]:
df.to_csv("./features/testing.csv",index=False)

In [13]:
print(df.head())

        0         1         2         3         4         5         6    \
0  0.322186  0.677898 -0.145250 -0.036973  0.213062  0.384598  0.466842   
1 -0.008387  0.963140 -0.160426 -0.069929  0.149145  0.117339  1.187987   
2  0.274111  0.673348 -0.116735  0.002396  0.043386  0.174210  0.807102   
3  0.010869  0.635782 -0.031681 -0.050103  0.204356  0.094646  1.132312   
4  0.222386  0.517263 -0.215346  0.188103  0.373835  0.502357  0.890878   

        7         8         9    ...       950       951       952       953  \
0 -0.157786 -0.077372  1.065681  ...  0.111386  0.815007  0.733062  0.037953   
1 -0.161798  0.037806  1.123000  ...  0.416256  0.691297  0.587217  0.080165   
2  0.017893 -0.199434  0.790279  ...  0.344022  0.645009  0.793129 -0.150971   
3  0.109250  0.016064  0.697127  ...  0.253762  0.463506  0.505665  0.165805   
4 -0.079455 -0.035185  0.761742  ...  0.286773  0.360084  0.549669  0.493099   

        954       955       956       957       958       959  
0  0